In [1]:
import rospy
import cv2
import numpy as np
from cv_bridge import CvBridge
from sensor_msgs.msg import Image

In [2]:
class SubscribeOnce:
    def __init__(self):
        rospy.init_node('subscribe_once', anonymous=True)
        self.bridge = CvBridge()
        self.sub = rospy.Subscriber('/coaxial_camera/inference/segmentation', Image, self.callback)
        self.subscribed_data = False
        
    def callback(self, data):
        self.subscribed_image = data
        self.cv_image = self.bridge.imgmsg_to_cv2(self.subscribed_image, "bgr8")
        self.sub.unregister()

In [3]:
def detect_red_color(img):
    """
    Detecting red color by filtering.
    Sometimes it detects yellow a little.
    """
    # convert to hsv
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # hsv 1
    hsv_min = np.array([0, 243, 50])
    hsv_max = np.array([30, 255, 255])
    mask1 = cv2.inRange(hsv_img, hsv_min, hsv_max)
    # hsv 2
    hsv_min = np.array([150, 243, 50])
    hsv_max = np.array([179, 255, 255])
    mask2 = cv2.inRange(hsv_img, hsv_min, hsv_max)
    # masking
    masked = mask1 + mask2
    
    return masked

In [4]:
node = SubscribeOnce()

In [5]:
image_sample = node.cv_image

In [11]:
bool_img = detect_red_color(image_sample)

In [13]:
bool_img.shape

(256, 320)

In [14]:
pixel_num = bool_img.shape[0] * bool_img.shape[1]

In [15]:
pixel_num

81920

In [18]:
np.sum(bool_img == 255)

13014

In [19]:
cv2.imshow("original", image_sample)
cv2.imshow("masked", detect_red_color(image_sample))
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [10]:
np.unique(detect_red_color(image_sample))

array([  0, 255], dtype=uint8)